In [5]:
import setproctitle
setproctitle.setproctitle('python4.1')  
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ExponentialLR
import sys
sys.path.append("..")
import utils.utils as utils
import utils.adv_ex_utils as aus
from utils.models import LeNet, DDNet, SimpleCNN
from utils.data_loaders import DataLoader
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack

# makes default tensor a CUDA tensor so GPU can be used
device = torch.device(1 if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

### Define train and test functions 

In [6]:
def train():
    net.train()
    
    for batch_idx, (samples, labels, target_interps) in enumerate(train_loader):
        # sends to GPU, i.e. essentially converts from torch.FloatTensor to torch.cuda.FloatTensor
        samples, labels, target_interps = samples.to(device), labels.to(device), target_interps.to(device)
#         samples = aus.perturb_randomly(samples, scale=.15, min=0., max=1.)
#         samples, labels = aus.generate_adv_exs(samples, labels, adversary)
                        
        optimizer.zero_grad()
        output = net(samples)
        loss = utils.interp_match_loss(output, labels, x=samples, target_interps=target_interps, 
                                       net=net, optimizer=optimizer, alpha=alpha)
#         loss = utils.my_loss(output, labels, net=net, optimizer=optimizer,
#                                         alpha_jr=alpha_jr, x=samples, bp_mat=tr)
#         loss = F.cross_entropy(output, labels)
#         loss = utils.jacobian_loss(output, labels, x=samples, alpha=alpha_jr, 
#                                    net=net, bp_mat=tr, optimizer=optimizer)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            j = utils.avg_norm_jacobian(net, samples, output.shape[1], tr, for_loss=False)
            i,_ = utils.norm_diff_interp(net, samples, labels, scale=.15, for_loss=False)
            print(f'\tLoss: {loss.item():.6f}, Average norm of Jacobian: {j:6f}, Norm of difference in interpretations: {i:6f}')
            train_losses.append(loss.item())
            jacobian_norms.append(j)
            interp_norm_diffs.append(i)

In [7]:
def test():
    net.eval()
    correct = 0
    adv_correct = 0
    
    for samples, labels, target_interps in test_loader:
        samples, labels, target_interps = samples.to(device), labels.to(device), target_interps.to(device)
        
        output = net(samples)
        
        if epoch == n_epochs:
            adv_samples, adv_labels = aus.generate_adv_exs(samples, labels, adversary, num_per_samp=2)
            adv_output = net(adv_samples)
            adv_preds = adv_output.data.max(1, keepdim=True)[1]
            adv_correct += adv_preds.eq(adv_labels.data.view_as(adv_preds)).sum().item()
        
        # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()
        
    if epoch == n_epochs: 
        adv_test_accuracy = 100. * float(adv_correct / (len(test_loader.dataset) * 2))
        print(f'\tPGD-perturbed test set accuracy: ({adv_test_accuracy:.2f}%)')
        
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'\tTest set accuracy: ({test_accuracy:.2f}%)')
    
    test_accuracies.append(test_accuracy)

### Training

In [ ]:
dataset = 'CIFAR-10'

# training details
n_epochs = 30
log_interval = 200
# torch.manual_seed(7)
# alphas = [0., 1e-3, 1e-2, 7e-2]
# alphas = [.75e-2, .75e-2, .75e-2, .75e-2]
alphas = [1e-1]*5
alphas = [2.5e-2]
# alpha = 7e-2
# alpha_jrs = [.015, .02, .025]
# alpha_jrs = [1.1e-3, 1.1e-3, 1.1e-3, 1.1e-3]
alpha_jrs = [2.8e-3, 2.8e-3, 2.8e-3, 2.8e-3]
# threshs = [None, -2., -1., 0., 1., 2., 2.5]
thresh=0

# dictionary to record each model's training/testing stats
performance = {}
q = 0

for alpha in alphas:
    dl = DataLoader(dataset='CIFAR-10_interps_stdtrain', tr_batch_size=100, te_batch_size=100, 
                    augment=False, path='../data', thresh=thresh, model='lenet')
#     dl = DataLoader(dataset='MNIST', tr_batch_size=64, te_batch_size=100, 
#                     augment=False, path='../data', thresh=thresh)
    train_loader = dl.train_loader
    test_loader = dl.test_loader
    tr_batch_size = dl.tr_batch_size
    te_batch_size = dl.te_batch_size
    # create matrices for back propagation
    tr = utils.bp_matrix(tr_batch_size, 10)
    te = utils.bp_matrix(te_batch_size, 10)
    # instantiate model and optimizer
    learning_rate = 0.01
    momentum = 0.9
    net = DDNet(p_drop=.5)
#     net = SimpleCNN(p_drop=.5)
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)#, weight_decay=.0005)
    lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)
#     lr_decayer = ExponentialLR(optimizer, gamma=.95)

    # make model CUDA enabled and define GPU/device to use
    net.cuda()

    # define adversary to train against if needed
    adversary = PGDAttack(predict=net, loss_fn=F.cross_entropy, eps=.314, 
                          nb_iter=40, eps_iter=.04, rand_init=True, 
                          clip_min=0., clip_max=1., ord=2, targeted=False)

    # for tracking training progress
    train_losses = []
    test_accuracies = []
    jacobian_norms = []
    interp_norm_diffs = []

    for epoch in range(1, n_epochs + 1):
        print(f'Epoch #{epoch}')
        train()
        test()
        lr_decayer.step()

    #     performance['simplecnn_alpha{}'.format(alpha)] = (train_losses, test_accuracies, 
    #                                               jacobian_norms, interp_norm_diffs)
#     torch.save(net.state_dict(), 
#                f'../trained_models/{dataset}/simplecnn_jr/model{q}')
    q+=1

    ### Write performance dictionary to text file

#     f = open(f'../trained_models/{dataset}/simplecnn_stdtrain/performance.txt','w')
#     f.write(str(performance))
#     f.close()

    # to read dictionary from file:
    # f = open(f'models/training_round_{training_round}_performance.txt','r')
    # d = eval(f.read())

Epoch #1
	Loss: 4.928511, Average norm of Jacobian: 0.358935, Norm of difference in interpretations: 0.003409
	Loss: 4.933461, Average norm of Jacobian: 2.488898, Norm of difference in interpretations: 0.014809
	Loss: 4.623224, Average norm of Jacobian: 27.271135, Norm of difference in interpretations: 0.175872
	Test set accuracy: (29.08%)
Epoch #2
	Loss: 4.573931, Average norm of Jacobian: 38.527721, Norm of difference in interpretations: 0.222470
	Loss: 4.153757, Average norm of Jacobian: 58.362129, Norm of difference in interpretations: 0.720090
	Loss: 4.106948, Average norm of Jacobian: 111.437485, Norm of difference in interpretations: 3.101839
	Test set accuracy: (34.26%)
Epoch #3
	Loss: 4.195321, Average norm of Jacobian: 152.696884, Norm of difference in interpretations: 1.755640
	Loss: 4.241650, Average norm of Jacobian: 242.431992, Norm of difference in interpretations: 3.294142
	Loss: 4.113476, Average norm of Jacobian: 314.152100, Norm of difference in interpretations: 1.50

In [10]:
torch.save(net.state_dict(), 
               f'../trained_models/{dataset}/ddnet_stdtrain/model{0}')